In [1]:
# Metadata included
from fastai.vision.all import *
from fastai.vision.widgets import *
from fastai.layers import *


# The Amazing Bear Classifier!

You need to know whether you're being chased by a dangerous grizzly, or a sweet teddy bear, and you need an answer *fast*? Then you've come to the right place. Take a pic of the potentially vicious killer, and click 'upload' to classify it. (Important: this only handles grizzly bears, black bears, and teddy bears. It will **not** give a sensible answer for polar bears, a bear market, a bear of a man, or hot dogs.

----

In [2]:
# Instruções de como resolver o travamento do file upload em
# https://github.com/jupyter-widgets/ipywidgets/issues/2522

# jupyter classic funcionou
# jupyter notebook --config="/Users/aolchik/.jupyter/jupyter_notebook_config.py" bear_classifier.ipynb

# resolver com VSCode (ainda não funcionando)
# resolver com Voila
#   voila --Voila.tornado_settings='{"websocket_max_message_size": 104857600}' bear_classifier.ipynb
#   voila bear_classifier.ipynb --Voila.tornado_settings="{'websocket_max_message_size': 209715200}"
#   voila bear_classifier.ipynb --tornado_settings websocket_max_message_size=209715200
#   voila bear_classifier.ipynb --VoilaConfiguration.tornado_settings="{'websocket_max_message_size': 209715200}"
# pesquisar: voila FileUpload does not work
# try streamlit https://docs.streamlit.io/
# try Solara
#   https://itnext.io/webapps-in-python-with-solara-a-streamlit-killer-ab6fcc7bf5d7
#   https://github.com/widgetti/solara
#   https://solara.dev/
#   How to run Solara from command line?
#     voila <notebook>
#   Does Solara has a file upload widget?
#     No, but you can use the one from ipywidgets.
#   Trying to render content generates a rate limiting error
#     Trying to setup rate limiting variables through metadata - https://voila.readthedocs.io/en/stable/customize.html
#     It seems not to be working as dark theme is not appied


In [3]:
# from voila.app import Voila, _, Dict

# class CustomVoila(Voila):
#   tornado_settings = Dict(
#     {
#       'websocket_max_message_size': 209715200
#     },
#     config=True,
#     help=_(
#         "Extra settings to apply to tornado application, e.g. headers, ssl, etc"
#     ),
#   )

# app = CustomVoila()

In [17]:
# print(app.tornado_settings)

{'websocket_max_message_size': 209715200}


In [8]:
# from ipyvuetify.extra import FileInput

# class CustomFileInput(FileInput):
#   def _file_info_changed(self, _):
#     display(widgets.Text(value='on_data_change called', disabled=True))
#     lbl_pred.value = ''
#     #img = PILImage.create(btn_upload.data[-1])
#     files = btn_upload.get_files()
#     img = PILImage.create(files[-1])
#     out_pl.clear_output()
#     with out_pl: display(img.to_thumb(128,128))
#     pred,pred_idx,probs = learn_inf.predict(img)
#     lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
#     super()._file_info_changed(_)

In [4]:
# path = Path()
# learn_inf = load_learner(path/'export.pkl', cpu=True)


In [ ]:
# btn_upload = widgets.FileUpload()
# # btn_upload = CustomFileInput()
# out_pl = widgets.Output()
# lbl_pred = widgets.Label()

In [13]:
import solara
import urllib.request

MODEL_FILENAME = "20231025_export.pkl"
MODEL_URL = "https://s3.amazonaws.com/projects.aolchik/bear_classifier/"+MODEL_FILENAME

@solara.component
def Page():
    # adding some css
    # solara.Style("""
    #     .add-button {
    #         margin-right: 10px;
    #     }
    # """)
    content, set_content = solara.use_state(b"")
    filename, set_filename = solara.use_state("")
    size, set_size = solara.use_state(0)
    status, set_status = solara.use_state("")
    prediction, set_prediction = solara.use_state("")
    path = Path()

    def on_file(file):
        set_filename(file["name"])
        set_size(file["size"])
        f = file["file_obj"]
        set_content(f.read())


    def predict():
        set_status("Checking if model is available...")
        if not (path/MODEL_FILENAME).exists():
            set_status(f"Downloading model {MODEL_FILENAME}...")
            urllib.request.urlretrieve(MODEL_URL, path/MODEL_FILENAME)

        learn_inf = load_learner(Path(".")/MODEL_FILENAME, cpu=True)

        img = PILImage.create(content)
        pred,pred_idx,probs = learn_inf.predict(img)
        set_prediction(f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}')
        return True

    with solara.Div(align="center") as main:
        with solara.Card(title="Select your bear"):
            solara.FileDrop(
                on_file = on_file
            )
            if content:
                solara.Info(f"File {filename} has total length: {size}\n, first 100 bytes:")
                solara.Image(content, format=filename.split(".")[-1])
            
                result: solara.Result[bool] = solara.use_thread(predict, dependencies=[content])

                if result.state == solara.ResultState.FINISHED:
                    if result.value:
                        solara.Preformatted(prediction)
                    else:
                        solara.Error(f"Could not predict successfully: {result.error} ")
                elif result.state == solara.ResultState.ERROR:
                    solara.Error(f"Error occurred: {result.error}")
                else:
                    solara.Info(f"Running... ({status})")
                    solara.ProgressLinear(True)


        

    return main

Page = Page()
Page

Cannot show ipywidgets in text

In [4]:
# files = btn_upload.get_files()
# files[-1]

In [3]:
# PILImage.create(files[-1]['file_obj'])

In [7]:
# def on_data_change(change):
#     display(widgets.Text(value='on_data_change called', disabled=True))
#     lbl_pred.value = ''
#     img = PILImage.create(btn_upload.data[-1])
#     out_pl.clear_output()
#     with out_pl: display(img.to_thumb(128,128))
#     pred,pred_idx,probs = learn_inf.predict(img)
#     lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

In [ ]:
#btn_upload.observe(on_data_change, names=['data'])

In [ ]:
# display(VBox([widgets.Label('Select your bear!'), btn_upload, out_pl, lbl_pred]))

In [6]:
# btn_upload.value
# btn_upload.get_files()

({'name': '2021-02-12 13.43.42.jpg',
  'type': 'image/jpeg',
  'size': 0,
  'content': <memory at 0x15e187d00>,
  'last_modified': datetime.datetime(2021, 2, 12, 16, 43, 43, tzinfo=datetime.timezone.utc)},)